#### Example: Tested Chunk size for context: 

In [23]:
def split_text(text, max_size):
    words = text.split()
    chunks = []
    current_chunk = ""

    for word in words:
        if len(current_chunk.encode('utf-8')) + len(word.encode('utf-8')) < max_size:
            current_chunk += word + " "
        else:
            chunks.append(current_chunk)
            current_chunk = word + " "
    if current_chunk:
        chunks.append(current_chunk)
    
    return chunks

test_text = """
BMW Bakım Paketleri, aracınız için bir dizi avantaj sunarak, araç bakımını hem daha ekonomik hem de daha rahat bir deneyim haline getiriyor. Bu paketler, bireysel bakım hizmetlerine kıyasla önemli maliyet tasarrufu sağlarken, aynı zamanda BMW'nizin ikinci el değerini de koruyor.

BMW Bakım Paketlerinin sunduğu avantajlar arasında öne çıkanlar şunlardır:

Maliyet Tasarrufu: Ayrı ayrı yapılan bakımlar yerine bu paketler ile toplamda daha az ödeme yaparak maliyet avantajından yararlanabilirsiniz.
Esnek Seçenekler: BMW'nizin ihtiyaçlarına ve bütçenize uygun farklı bakım paketleri arasından seçim yapma özgürlüğü sunar.
Değer Koruma: Araç bakım geçmişinin eksiksiz olması, ikinci el piyasasında BMW'nizin değerini korumanıza yardımcı olur.
Anında Erişim ve Kullanım: Online olarak hızlı ve kolay bir şekilde satın alınabilen paketler, ihtiyaç duyduğunuz anda hemen kullanılabilir.
Maliyet Kontrolü: Yıllar içindeki potansiyel fiyat artışlarından etkilenmeden, harcamalarınızı önceden belirlenen tutarlarla kontrol altında tutabilirsiniz.
Devir Kolaylığı: BMW'nizi satmanız durumunda, bakım paketi otomatik olarak yeni sahibine geçerek aracın değerini korur.
Kanıtlanmış Kalite: BMW Bakım Paketleri, yalnızca BMW Orijinal Parçaları ve motor yağlarını kullanır, bu da BMW'nizin yüksek teknik standartlarını korumanıza yardımcı olur.
Geniş Geçerlilik: Bu paketler, sadece Türkiye'de değil, dünya çapında birçok BMW Yetkili Servisi'nde geçerlidir.
Kısacası, BMW Bakım Paketleri, hem maliyet etkinliği hem de aracınızın değerini koruma açısından önemli faydalar sunarak, BMW sahiplerine önemli kolaylıklar sağlamaktadır. Bu paketler, araç bakımını hem daha planlanabilir hem de daha ekonomik hale getirerek, BMW sürüş deneyiminizi daha da keyifli kılmayı hedefliyor.
""" 
chunks = split_text(test_text, 1400)

chunk_sizes = [len(chunk.encode('utf-8')) for chunk in chunks]
chunk_sizes, chunks

([1385, 511],
 ["BMW Bakım Paketleri, aracınız için bir dizi avantaj sunarak, araç bakımını hem daha ekonomik hem de daha rahat bir deneyim haline getiriyor. Bu paketler, bireysel bakım hizmetlerine kıyasla önemli maliyet tasarrufu sağlarken, aynı zamanda BMW'nizin ikinci el değerini de koruyor. BMW Bakım Paketlerinin sunduğu avantajlar arasında öne çıkanlar şunlardır: Maliyet Tasarrufu: Ayrı ayrı yapılan bakımlar yerine bu paketler ile toplamda daha az ödeme yaparak maliyet avantajından yararlanabilirsiniz. Esnek Seçenekler: BMW'nizin ihtiyaçlarına ve bütçenize uygun farklı bakım paketleri arasından seçim yapma özgürlüğü sunar. Değer Koruma: Araç bakım geçmişinin eksiksiz olması, ikinci el piyasasında BMW'nizin değerini korumanıza yardımcı olur. Anında Erişim ve Kullanım: Online olarak hızlı ve kolay bir şekilde satın alınabilen paketler, ihtiyaç duyduğunuz anda hemen kullanılabilir. Maliyet Kontrolü: Yıllar içindeki potansiyel fiyat artışlarından etkilenmeden, harcamalarınızı önceden

#### Download required library -> Langchain

In [ ]:
!pip install langchain

#### see langchain version below (for reference)

In [112]:
import langchain 
print(langchain.__version__)

0.0.350


##### The **Borusan** -> AutoHackPDFExtractor class is designed to extract and segment text from PDFs into manageable chunks with specified overlap for efficient processing during the Borusan Auto Hackathon.

In [75]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

class AutoHackPDFExtractor:
    def __init__(self, file_path, chunk_size=2300, overlap_size=100):
        """
        Initializes the BorusanHackathonPDFExtractor.

        Args:
            file_path (str): The path to the PDF file for the Borusan Hackathon.
            chunk_size (int): Maximum size of each text chunk.
            overlap_size (int): Size of the overlap between chunks.
        """
        self.file_path = file_path
        self.chunk_size = chunk_size
        self.overlap_size = overlap_size
        self.full_text = ""
        self.chunks = []
        self.pdf_loader = PyPDFLoader(self.file_path)
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.overlap_size,
            length_function=len  
        )

    def extract_text(self):
        """Extracts text from the PDF file."""
        extracted_documents = self.pdf_loader.load()
        # Extract text from the 'page_content' attribute of each Document
        self.full_text = " ".join([doc.page_content for doc in extracted_documents])

    def split_into_chunks(self):
        """Splits the extracted text into chunks with overlap."""
        documents = [self.full_text]
        split_documents = self.text_splitter.create_documents(documents)
        self.chunks = [doc.page_content for doc in split_documents]

    def get_chunks(self):
        """Returns the list of text chunks."""
        return self.chunks

    def process_pdf(self):
        """Processes the PDF file to extract and split text."""
        self.extract_text()
        self.split_into_chunks()

### Get chunks

In [76]:
extractor = AutoHackPDFExtractor("source/BorusanAutoHack.pdf")
extractor.process_pdf()
chunks = extractor.get_chunks()

In [77]:
text = chunks[0]
text_size = len(text.encode('utf-8'))

print("-"*50)

print(f"Text size: {text_size}")

print("-"*50)

print("*"*100)

print(text)

print("*"*100)

--------------------------------------------------
Text size: 2443
--------------------------------------------------
****************************************************************************************************
Borusan Oto, BMW araç sahiplerine sunduğu geniş kapsamlı satış sonrası hizmetleri ile dikkat 
çekiyor : BMW Türkiye Mobil Uygulaması: Bu yenilikçi uygulama,  hem potansiyel BMW alıcılarına hem 
de mevcut sürücülere araç kullanımının tüm yönleriyle ilgili 360 derece özellikler sunuyor. Kullanıcılar, 
bu uygulama sayesinde araçlarının performansını izleyebilir ve çeşitli hizmetlere kolayca erişebilirler.  
Yol Yardı m Hizmeti: BMW sürücüleri, herhangi bir arıza veya kaza durumunda 7 gün 24 saat 
erişilebilen Acil Servis hizmetinden yararlanabilirler. 444 22 99 numaralı Yol Yardım Hattı üzerinden 
ulaşılan profesyonel ekip, araçlara yerinde müdahale edebilir veya Yetkili  Servise çekilmesini 
sağlayabilir.  BMW Service Inclusive: Bu paket, BMW'nizin bakımını belirlenen süre

## Download required libraries

In [ ]:
!pip install openai python-dotenv
!pip install --upgrade qdrant_client

#### see openai version below (for reference)

In [79]:
import openai 
print(openai.VERSION)

1.3.9


### Initialize Azure OpenAI client with API credentials loaded from environment variables.

In [88]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv() 

API_KEY = os.getenv('AZURE_OPENAI_EMBD_API_KEY')
ENDPOINT = os.getenv('AZURE_OPENAI_EMBD_ENDPOINT')

azure_client = AzureOpenAI(
  api_key=API_KEY,
  api_version="2023-05-15",
  azure_endpoint=ENDPOINT
)

##### Configuration for the embedding model: model name - openai embedding model

In [89]:
embedding_model = "text-embedding-ada-002" 

##### Convert text chunks to PointStructs with embeddings for Qdrant indexing.

In [93]:
from qdrant_client.http.models import PointStruct

def create_point_structs_from_chunks(chunks):
    """
    Verilen metin parçalarından (chunks) PointStruct nesneleri oluşturur.
    
    Args:
    chunks (list of str): Metin parçalarının listesi.

    Returns:
    list of PointStruct: Oluşturulan PointStruct nesnelerinin listesi.
    """
    points = []

    for index, chunk in enumerate(chunks, start=1):
        embeddings = azure_client.embeddings.create(input=chunk, model=embedding_model).data[0].embedding
        points.append(PointStruct(id=index, vector=embeddings, payload={"text": chunk}))

    return points

#### Get points

In [94]:
points = create_point_structs_from_chunks(chunks)  

#####  Create a client instance for Qdrant

In [95]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333) 

#### Create collection with vector configuration - Cosine distance and 1536 vector size

In [98]:
from qdrant_client.http.models import VectorParams, Distance

client.recreate_collection(
    collection_name="BorusanAutoHack",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

####  Upsert points to the collection

In [100]:
client.upsert(
    collection_name="BorusanAutoHack",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

#### see collection info or open http://localhost:6333/collections/AutoHackathon in browser for more details

In [102]:
collection_info = client.get_collection(collection_name="AutoHackathon")
print("Number of vectors in the Coderspace - Auto Hackathon collection:", collection_info.vectors_count)

Number of vectors in the Coderspace - Auto Hackathon collection: 13


#### Query Qdrant 

In [108]:
def query_qdrant(query, top_k=5):

    embedded_query = azure_client.embeddings.create(input=query, model=embedding_model).data[0].embedding

    qdrant_search_result = client.search(
        collection_name="AutoHackathon",
        query_vector=embedded_query, 
        limit=top_k
    )

    return qdrant_search_result

#### Question: 

In [109]:
question = "Satış sonrası hangi hizmetler sunulmaktadır?"

#### Print result

In [110]:
import json 

def print_search_results_json(results):
    for i, result in enumerate(results):
        result_dict = {
            'Result': i + 1,
            'Score': result.score,
            'Query Result': result.payload['text']
        }

        print(json.dumps(result_dict, indent=4, ensure_ascii=False))

print_search_results_json(query_qdrant(question))

{
    "Result": 1,
    "Score": 0.8160229,
    "Query Result": "Borusan Oto, BMW araç sahiplerine sunduğu geniş kapsamlı satış sonrası hizmetleri ile dikkat \nçekiyor : BMW Türkiye Mobil Uygulaması: Bu yenilikçi uygulama,  hem potansiyel BMW alıcılarına hem \nde mevcut sürücülere araç kullanımının tüm yönleriyle ilgili 360 derece özellikler sunuyor. Kullanıcılar, \nbu uygulama sayesinde araçlarının performansını izleyebilir ve çeşitli hizmetlere kolayca erişebilirler.  \nYol Yardı m Hizmeti: BMW sürücüleri, herhangi bir arıza veya kaza durumunda 7 gün 24 saat \nerişilebilen Acil Servis hizmetinden yararlanabilirler. 444 22 99 numaralı Yol Yardım Hattı üzerinden \nulaşılan profesyonel ekip, araçlara yerinde müdahale edebilir veya Yetkili  Servise çekilmesini \nsağlayabilir.  BMW Service Inclusive: Bu paket, BMW'nizin bakımını belirlenen süre ve kilometre \nsınırları dahilinde, dünya genelindeki BMW Yetkili Servisleri tarafından gerçekleştirmeyi garanti eder. \nBu sayede, araç sahipleri 